In [1]:
!pip install -U pip
!pip install pandas==1.0.1
!pip install numpy==1.18.1
!pip install imblearn==0.0
!pip install xgboost==1.2.1

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (20.2.4)


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE

In [3]:
# Seed randômico para garantir reprodutibilidade
# em relação à base amostral
SEED=42

In [4]:
# Carga dos dados e ajuste das colunas
df_bytebank = pd.read_csv('dados/bytebank_credito.csv')

df_bytebank.columns \
    = [
        'ID',
        'CREDITO_CONCEDIDO',
        'GENERO',
        'ESCOLARIDADE',
        'ESTADO_CIVIL',
        'IDADE',
        'ATRASO_PARCELA_M_0',
        'ATRASO_PARCELA_M_2',
        'ATRASO_PARCELA_M_3',
        'ATRASO_PARCELA_M_4',
        'ATRASO_PARCELA_M_5',
        'ATRASO_PARCELA_M_6',
        'SALDO_PAGO_PARCELA_1',
        'SALDO_PAGO_PARCELA_2',
        'SALDO_PAGO_PARCELA_3',
        'SALDO_PAGO_PARCELA_4',
        'SALDO_PAGO_PARCELA_5',
        'SALDO_PAGO_PARCELA_6',
        'VALOR_PAGAMENTO_ANTERIOR_1',
        'VALOR_PAGAMENTO_ANTERIOR_2',
        'VALOR_PAGAMENTO_ANTERIOR_3',
        'VALOR_PAGAMENTO_ANTERIOR_4',
        'VALOR_PAGAMENTO_ANTERIOR_5',
        'VALOR_PAGAMENTO_ANTERIOR_6',
        'INADIMPLENTE'
    ]

In [5]:
# Vamos remover a coluna ID pois ela não 
# não será necessária no nosso treinamento
del df_bytebank['ID']

In [6]:
# Proporção de dados de inadimplência
df_bytebank.groupby(['INADIMPLENTE']).size()

INADIMPLENTE
0    23364
1     6636
dtype: int64

In [7]:
# Geração de conjunto de dados para criação de
# novos registros (oversampling)
X = df_bytebank.iloc[:, 0:23]
y = df_bytebank.INADIMPLENTE

In [8]:
# Vamos fazer a criação de registros
# para a classe de inadimplentes
# INADIMPLENTE=1 e vamos usar o 
# metodo "fit_resample" para criar as amostras
oversample = SMOTE(sampling_strategy ='minority',
                   n_jobs=-1,
                   random_state=SEED)

X, y = oversample.fit_resample(X, y)

In [9]:
# Vamos checar as proporções da nossa base
Counter(y)

Counter({1: 23364, 0: 23364})

In [10]:
# Vamos colocar a coluna "INADIMPLENTE" na primeira posição 
# do dataframe. Esse é o formato no qual o Sage Maker trabalha
# para reaizar os ajustes (fit) do modelo
y = pd.DataFrame(y)
y.columns = ['INADIMPLENTE']

df_bytebank = \
    pd.concat([y,
               X], axis=1)

In [11]:
# Embaralhamento da base de dados
df_bytebank = df_bytebank.sample(frac=1)

In [12]:
# Geração de array randomico 
# para escolha via de partes 
# do dataframe
np.random.seed(SEED)
probabilidade = np.random.rand(len(df_bytebank))

treino_registros = probabilidade < 0.7
teste_registros = (probabilidade >= 0.7) & (probabilidade < 0.85)
validacao_registros = probabilidade >= 0.85

df_bytebank_treino = df_bytebank[treino_registros]
df_bytebank_teste = df_bytebank[teste_registros]
df_bytebank_validacao = df_bytebank[validacao_registros]

# Vamos salvar localmente os arquivos em CSV
# já realizando a remoção do índice
df_bytebank_treino.to_csv('dados/df_bytebank_treino.csv', index=None)
df_bytebank_teste.to_csv('dados/df_bytebank_teste.csv', index=None)
df_bytebank_validacao.to_csv('dados/df_bytebank_validacao.csv', index=None)

### Referências

[SMOTE with Imbalance Data](https://www.kaggle.com/qianchao/smote-with-imbalance-data)

[How do I create test and train samples from one dataframe with pandas?](https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas)

[choosing features with random forest (80% acc)](https://www.kaggle.com/drproduck/choosing-features-with-random-forest-80-acc)

[SMOTE for Imbalanced Classification with Python](https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)